# Parquet files with column mapping enabled

In [1]:
import pyspark
from delta import *

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3eab10ca-db02-4ec5-8772-ac978bb86612;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 387ms :: artifacts dl 27ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number|

22/09/19 13:36:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/19 13:36:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
columns = ["language", "num_speakers"]
data = [("English", "1.5"), ("Mandarin", "1.1"), ("Hindi", "0.6")]
rdd = spark.sparkContext.parallelize(data)
df = rdd.toDF(columns)

In [7]:
df.write.format("delta").saveAsTable("mydeltalake1")

In [8]:
spark.sql(
    """ALTER TABLE `mydeltalake1` SET TBLPROPERTIES (
   'delta.columnMapping.mode' = 'name',
   'delta.minReaderVersion' = '2',
   'delta.minWriterVersion' = '5')"""
)

DataFrame[]

In [9]:
!tree spark-warehouse/mydeltalake1

spark-warehouse/mydeltalake1
├── _delta_log
│   ├── 00000000000000000000.json
│   └── 00000000000000000001.json
├── part-00000-1cfa9924-48d5-4573-abbe-db4531926c64-c000.snappy.parquet
├── part-00003-054d4636-fd36-45c9-aec8-62d79d2ff260-c000.snappy.parquet
├── part-00006-84743340-f7db-4560-9d4a-6b952f69762a-c000.snappy.parquet
└── part-00009-f1827133-315f-4e46-ae06-d61601435aab-c000.snappy.parquet

1 directory, 6 files


## Inspect Parquet file with PyArrow

In [10]:
import pyarrow.parquet as pq

In [12]:
parquet_file = pq.ParquetFile('spark-warehouse/mydeltalake1/part-00000-1cfa9924-48d5-4573-abbe-db4531926c64-c000.snappy.parquet')

In [14]:
parquet_file.schema

required group field_id=-1 spark_schema {
  optional binary field_id=-1 language (String);
  optional binary field_id=-1 num_speakers (String);
}